In [1]:
from modules import data_collection, models, optimizer, data_cleaning
import pandas as pd
import _pickle as cPickle

In [2]:
with open(r"../Data/FBREF_22_23.pickle", "rb") as inputfile:
    df = cPickle.load(inputfile)

In [3]:
search, y_pred, mse, rmse, r2 = models.model(df.drop(["Player","Cost"], axis = 1))

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


In [4]:
for i in [mse, rmse, r2]:
    print(i)

274.5887225594264
16.570718830498162
0.44440420707976236


In [5]:
expected_scores = search.best_estimator_.predict(df.drop(["Player","Cost"], axis = 1))
prices = (df.Cost / 10).to_list()
position = df.Position.map({'GK':1, 'DEF':2, 'MID':3, 'FWD':4}).to_list()
team = df.Squad.to_list()
names = df.Player.to_list()

In [7]:
import importlib as imp

imp.reload(optimizer)

decisions, captain_decisions, sub_decisions = optimizer.select_team(
    expected_scores, prices, position, team
)

325
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/caleb/PythonEnvs/FPL/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/2e7806352b3c4a9a9b2df99063ab3e19-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/2e7806352b3c4a9a9b2df99063ab3e19-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 689 COLUMNS
At line 8141 RHS
At line 8826 BOUNDS
At line 9802 ENDATA
Problem MODEL has 684 rows, 975 columns and 4526 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 867.191 - 0.00 seconds
Cgl0004I processed model has 680 rows, 975 columns (975 integer (975 of which binary)) and 4223 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.571429
Cbc0038I Pass   1: suminf.    0.27027 (2) obj. -861.215 iterations 15
Cbc0038I Solution found of -810.73
Cbc0038I Before mini branch and bound, 972 integers at bo

In [8]:
pos_map = {1:'GK', 2:'DEF', 3: 'MID', 4:'FWD'}
team_names = []
for i in range(len(decisions)):
    if decisions[i].value() != 0:
        team_names.append(names[i])
        print(f'{names[i]} {pos_map[position[i]]}: {expected_scores[i]} points at ${prices[i]}')
print('subs')
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() != 0:
        team_names.append(names[i])
        print(f'{names[i]} {pos_map[position[i]]}: {expected_scores[i]} points at ${prices[i]}')

Timothy Castagne DEF: 49.77616882324219 points at $4.7
Kevin De Bruyne MID: 82.77982330322266 points at $12.6
Roberto Firmino FWD: 56.38548278808594 points at $8.1
James Maddison MID: 73.58306121826172 points at $8.2
Mason Mount MID: 73.53397369384766 points at $7.5
Nick Pope GK: 58.36415481567383 points at $5.3
Fabian Schär DEF: 53.404273986816406 points at $4.9
Ivan Toney FWD: 74.33267211914062 points at $7.4
Kieran Trippier DEF: 75.52534484863281 points at $5.9
Leandro Trossard MID: 72.80774688720703 points at $7.1
Martin Ødegaard MID: 73.09465026855469 points at $6.4
subs
Eric Dier DEF: 47.24419021606445 points at $5.2
Ben Mee DEF: 45.99644088745117 points at $4.6
Jordan Pickford GK: 51.75877380371094 points at $4.5
Callum Wilson FWD: 47.91029739379883 points at $7.4
